<a href="https://colab.research.google.com/github/whyranox3/Hacksa/blob/master/NewFityou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 불러오기
from IPython.display import Image as IPyImage, display
from PIL import Image
import os
import torch
from torch.autograd import Variable
import numpy as np
from torchvision import transforms
from google.colab import files

In [ ]:
# 사용자 입력 받기
height = int(input("키(cm)를 입력하세요: "))
weight = int(input("몸무게(kg)를 입력하세요: "))

In [ ]:
# 키 구간
if 150 <= height < 160:
    height_code = 'A'
elif 160 <= height < 170:
    height_code = 'B'
elif 170 <= height < 180:
    height_code = 'C'
else:
    height_code = 'C'  # 예외 처리

In [ ]:
# BMI 계산
height_m = height / 100
bmi = weight / (height_m ** 2)

In [ ]:
# 체중 구간
if bmi < 18.5:
    weight_code = 'under'
elif bmi < 25:
    weight_code = 'normal'
else:
    weight_code = 'over'

In [ ]:
# 아바타 이미지 선택
filename = f"{height_code}_{weight_code}.jpg"
avatar_path = f"/content/{filename}"
print(f"체형: {filename}")

avatar = Image.open(avatar_path).convert("RGBA")
display(avatar)

In [ ]:
# U-2-Net 다운로드 및 준비
!git clone https://github.com/xuebinqin/U-2-Net
%cd U-2-Net
!pip install gdown opencv-python pillow numpy

%cd saved_models
!gdown --id 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
%cd ..

In [ ]:
# 옷 저장 디렉토리 준비
upper_dir = "outputs/upper_clothes"
lower_dir = "outputs/lower_clothes"
os.makedirs(upper_dir, exist_ok=True)
os.makedirs(lower_dir, exist_ok=True)

In [ ]:
# 상의 업로드
print("상의 이미지를 업로드해주세요")
uploaded_upper = files.upload()
upper_filename = "shirt.jpg"

# 업로드된 상의 파일 경로 추출
upper_input_path = None # 나중에 누끼 처리에 사용
for upper_filename in uploaded_upper:
    upper_input_path = os.path.join(upper_dir, upper_filename)
    Image.open(upper_filename).save(upper_input_path)
    print(f"상의 저장 완료: {upper_input_path}")

In [ ]:
# 하의 업로드
print("하의 이미지를 업로드해주세요")
uploaded_lower = files.upload()
lower_filename = "pants.jpg"

for fname in uploaded_lower:
    lower_input_path = os.path.join(lower_dir, fname)
    Image.open(fname).save(lower_input_path)
    print(f"하의 저장 완료: {lower_input_path}")

In [ ]:
# U-2-Net 관련 모듈
from model import U2NETP
from u2net_test import normPRED, save_output
from data_loader import RescaleT, ToTensorLab, ToTensor

# U-2-Net 누끼 처리 함수
def process_with_u2net(input_path, output_dir, model_name='u2net'):
  print(f"[U-2-Net 처리 시작] {input_path}")

In [ ]:
# 모델 로딩
model_dir = 'saved_models/u2netp.pth'
net = U2NETP(3, 1)
net.load_state_dict(torch.load(model_dir, map_location='cpu'))
net.eval()

In [ ]:
#누끼 상/하의 통합 함수
def remove_background(input_path: str, output_dir: str, net, label: str = "image") -> str:
    """
    U-2-Net을 사용하여 배경 제거된 이미지를 생성하고 저장합니다.

    Args:
        input_path (str): 원본 이미지 경로
        output_dir (str): 결과 이미지 저장 폴더
        net: 학습된 U2NETP 모델 객체
        label (str): 출력 파일명에 붙일 라벨 ("image" 대신 "upper", "lower" 등 사용 가능)

    Returns:
        str: 저장된 이미지 경로
    """
    # 이미지 열기 및 전처리
    image_np = np.array(Image.open(input_path).convert('RGB'))
    transform = transforms.Compose([
        RescaleT(320),
        ToTensorLab(flag=0)
    ])
    sample = transform({
        'imidx': np.array([0]),
        'image': image_np,
        'label': image_np
    })
    inputs_test = Variable(torch.unsqueeze(sample['image'], 0)).type(torch.FloatTensor)

    # 예측 실행
    with torch.no_grad():
        d1, *_ = net(inputs_test)
        pred = d1[:, 0, :, :]
        pred = (pred - pred.min()) / (pred.max() - pred.min())
        mask = (pred.detach().numpy()[0] * 255).astype('uint8')

    # 알파 채널 적용
    img_resized = Image.fromarray(image_np).resize((mask.shape[1], mask.shape[0]))
    img_np = np.array(img_resized)
    rgba = np.dstack((img_np, mask))

    # 흰 배경 투명 처리
    r, g, b, a = rgba[:, :, 0], rgba[:, :, 1], rgba[:, :, 2], rgba[:, :, 3]
    white_areas = (r > 240) & (g > 240) & (b > 240)
    rgba[white_areas, 3] = 0

    # 저장
    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.splitext(os.path.basename(input_path))[0]
    out_file = os.path.join(output_dir, f"seg_clean_{label}_{filename}.png")
    Image.fromarray(rgba).save(out_file)

    print(f"[✅ 누끼 이미지 저장 완료] → {out_file}")
    return out_file


In [ ]:
# 모델 로딩
from model import U2NETP
net = U2NETP(3, 1)
net.load_state_dict(torch.load("saved_models/u2netp.pth", map_location='cpu'))
net.eval()

# 상의 누끼
upper_output = "outputs/upper_clothes"
remove_background(upper_input_path, upper_output, net, label="upper")

# 하의 누끼
lower_output = "outputs/lower_clothes"
remove_background(lower_input_path, lower_output, net, label="lower")


In [ ]:
# 세그멘테이션 함수 정의
def generate_mask(image_path, output_dir, label="upper"):
    # 출력 디렉토리 준비
    os.makedirs(output_dir, exist_ok=True)

    # 이미지 로드 및 전처리
    image_np = np.array(Image.open(image_path).convert('RGB'))
    transform = transforms.Compose([
        RescaleT(320),
        ToTensorLab(flag=0)
    ])
    sample = transform({'imidx': np.array([0]), 'image': image_np, 'label': image_np})
    image_tensor = Variable(torch.unsqueeze(sample['image'], 0)).type(torch.FloatTensor)

    # 모델 추론
    with torch.no_grad():
        d1, *_ = net(image_tensor)
        pred = d1[:, 0, :, :]
        pred = (pred - pred.min()) / (pred.max() - pred.min())  # 정규화

    # 마스크 저장 (흑백 PNG)
    mask_np = (pred[0].cpu().numpy() * 255).astype(np.uint8)
    mask_img = Image.fromarray(mask_np).resize(Image.open(image_path).size)

    # 저장 경로 지정
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    save_path = os.path.join(output_dir, f"{base_name}_{label}_mask.png")
    mask_img.save(save_path)

    print(f"✅ 마스크 저장 완료: {save_path}")
    return save_path

In [ ]:
# 🔻 상의 마스크 생성
generate_mask(
    image_path=avatar_path,
    output_dir="outputs/avatar_upper_mask",
    label="upper"
)

# 🔻 하의 마스크 생성
generate_mask(
    image_path=avatar_path,
    output_dir="outputs/avatar_lower_mask",
    label="lower"
)
#마스크 dir 새로 생성됨

In [ ]:
#의상 합성 함수
def apply_clothes_to_avatar(avatar_path, clothing_rgba_path, mask_path, output_path):
    """
    아바타에 의상을 입히는 함수 (마스크 기준으로 합성)
    """

    # 아바타 이미지 (RGB), 옷 이미지 (RGBA), 마스크 이미지 (L)
    avatar = Image.open(avatar_path).convert("RGBA")
    clothes = Image.open(clothing_rgba_path).convert("RGBA")
    mask = Image.open(mask_path).convert("L")  # 흑백 마스크

    # 모두 같은 크기로 리사이즈
    clothes = clothes.resize(avatar.size)
    mask = mask.resize(avatar.size)

    # 마스크를 통해 옷 이미지 적용할 영역만 남기기
    clothes_np = np.array(clothes)
    mask_np = np.array(mask)

    # 마스크가 흰색(255)인 부분만 의상을 살리고 나머지는 투명
    alpha_mask = (mask_np > 128).astype(np.uint8)  # True/False → 1/0
    for c in range(3):  # RGB 채널
        clothes_np[:, :, c] = clothes_np[:, :, c] * alpha_mask
    clothes_np[:, :, 3] = clothes_np[:, :, 3] * alpha_mask  # 알파 채널도 마찬가지

    clothes_masked = Image.fromarray(clothes_np)

    # 아바타와 의상 합성
    combined = Image.alpha_composite(avatar, clothes_masked)

    # 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    combined.save(output_path)
    print(f"🎉 의상 합성 완료: {output_path}")

In [ ]:
#의상 합성 함수2...
def get_filename_without_ext(path):
    return os.path.splitext(os.path.basename(path))[0]

def auto_apply_clothes(
    avatar_path,
    upper_rgba_path,
    lower_rgba_path,
    upper_mask_dir="U-2-Net/outputs/avatar_upper_mask",
    lower_mask_dir="U-2-Net/outputs/avatar_lower_mask",
    result_dir="results"
):
    os.makedirs(result_dir, exist_ok=True)

    avatar_name = get_filename_without_ext(avatar_path)

    # ✅ 상의 마스크 경로
    upper_mask_path = f"outputs/avatar_upper_mask/{avatar_name}_upper_mask.png"
    if not os.path.exists(upper_mask_path):
        print(f"[❌오류] 상의 마스크 없음: {upper_mask_path}")
        return

    # ✅ 하의 마스크 경로
    lower_mask_path = f"outputs/avatar_lower_mask/{avatar_name}_lower_mask.png"
    if not os.path.exists(lower_mask_path):
        print(f"[❌오류] 하의 마스크 없음: {lower_mask_path}")
        return

    # ✅ 상의 합성
    upper_output_path = os.path.join(result_dir, f"{avatar_name}_with_upper.png")
    apply_clothes_to_avatar(
        avatar_path=avatar_path,
        clothing_rgba_path=upper_rgba_path,
        mask_path=upper_mask_path,
        output_path=upper_output_path
    )
    print(f"[✅ 상의 합성 완료] → {upper_output_path}")

    # ✅ 하의 합성
    final_output_path = os.path.join(result_dir, f"{avatar_name}_with_full_outfit.png")
    apply_clothes_to_avatar(
        avatar_path=upper_output_path,
        clothing_rgba_path=lower_rgba_path,
        mask_path=lower_mask_path,
        output_path=final_output_path
    )
    print(f"[✅ 하의 합성 완료] → {final_output_path}")

    # ✅ 최종 결과 이미지 출력
    try:
        result_img = Image.open(final_output_path)
        display(result_img)
    except Exception as e:
        print(f"[❌오류] 최종 이미지 표시 실패: {e}")


In [ ]:
auto_apply_clothes(
    avatar_path = avatar_path,
    upper_rgba_path="outputs/upper_clothes/seg_clean_shirt.png",
    lower_rgba_path="outputs/lower_clothes/seg_clean_pants.png"
)